创建数据的neo4j Cypher语句：

```
CREATE (a1:很搞笑的理由 {值:'游回来'}) - [:属于 {前提条件_当证券代码等于:'002069'}] -> (b1:存在异常 {值:'很搞笑的理由'}) <-  [:属于 {前提条件_当证券代码等于:'002069'}] - (c1:很搞笑的理由 {值:'逃跑'});

match (x:很搞笑的理由 {值:'逃跑'})
CREATE (a2:输入词 {值:'死亡'}) - [:归类为 {前提条件_当证券代码等于:'002069'}] -> (x) <-  [:归类为 {前提条件_当证券代码等于:'002069'}] - (c2:输入词 {值:'异常'});

match data=(a3:存在异常) -[*]-(b3)
return data;
```

In [1]:
try:
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
except:
    %pip install pdfminer
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage

In [2]:
from io import StringIO

def 读取pdf(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
#遍历pdf目录找出所有pdf文件
import os
def 遍历pdf目录找出所有pdf文件(folder):
    file_list = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.lower().endswith('.pdf'):
                file_list.append(os.path.join(root, file))
    return file_list

In [4]:
# 获取当前目录
import os
def 获取当前目录():
    return os.path.dirname(os.path.abspath(__file__))

pdf文件列表 = 遍历pdf目录找出所有pdf文件('pdf')
pdf文件列表

['pdf\\2014年10月30日 关于海洋牧场灾情说明会的公告.PDF',
 'pdf\\2018年2月05日 关于底播虾夷扇贝2017年终盘点情况的公告.PDF',
 'pdf\\2019年11月11日 关于2019年秋季底播虾夷扇贝存量抽测的风险提示公告.PDF',
 'pdf\\第五届董事会第十七次会议决议公告.PDF']

In [5]:
def 获取pdf文件的文件名(pdf文件):
    pdf文件名 = pdf文件.split('\\')[-1]
    return pdf文件名

In [6]:
获取pdf文件的文件名(pdf文件列表[0])

'2014年10月30日 关于海洋牧场灾情说明会的公告.PDF'

Neo4j数据库操作相关

In [7]:
from neo4j import GraphDatabase

uri = "neo4j+s://1144f8d0.databases.neo4j.io:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "VPrCeTO-8CzYElUogkj45D4Ec0x454oBgAqHIjHZG4k")) #enter your neo4j username and password instead of 'test'
session = driver.session()

#define relevant functions to execute differnet queries
def run_query(tx, query, 证券代码,输入词列表):
    result_list=[]
    for record in tx.run(query, 证券代码=证券代码,输入词列表=输入词列表):
        result_list.append(record[0])
    return result_list

知识推理

In [8]:
图数据库查询用语句 = ("match (a:存在异常) -[b:属于 {前提条件_当证券代码等于:$证券代码}]-(c)-[d:归类为 {前提条件_当证券代码等于:$证券代码}] -(e:输入词) where e.值 in $输入词列表 return a")

可解释相关

In [9]:
# https://www.w3school.com.cn/python/ref_string_format.asp
可解释信息 = "因为该上市公司于{上次发生日期}发生过类似的事情，参考信息：{参考信息链接列表}".format(上次发生日期 = "2014年10月30日", 参考信息链接列表 = "https://zhuanlan.zhihu.com/p/382126195;https://www.sohu.com/a/474688580_473133")
可解释信息

'因为该上市公司于2014年10月30日发生过类似的事情，参考信息：https://zhuanlan.zhihu.com/p/382126195;https://www.sohu.com/a/474688580_473133'

In [10]:
证券代码列表 = ["002069"]

NLP相关

In [11]:
print('加载nlp模型')
import spacy
nlp = spacy.load('zh_core_web_sm')

加载nlp模型


c:\Users\billy\Desktop\demo\Hands-On-Artificial-Intelligence-for-Banking-Chinese\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
ruler = nlp.add_pipe("entity_ruler",config={"validate": True,"overwrite_ents": True})
patterns = [{"label": "证券代码", "pattern": code} for code in 证券代码列表]
ruler.add_patterns(patterns)

c:\Users\billy\Desktop\demo\Hands-On-Artificial-Intelligence-for-Banking-Chinese\env\lib\site-packages\spacy\pipeline\entityruler.py:334: UserWarning: [W012] A Doc object you're adding to the PhraseMatcher for pattern '证券代码' is parsed and/or tagged, but to match on 'ORTH', you don't actually need this information. This means that creating the patterns is potentially much slower, because all pipeline components are applied. To only create tokenized Doc objects, try using `nlp.make_doc(text)` or process all texts as a stream using `list(nlp.tokenizer.pipe(all_texts))`.
  self.phrase_matcher.add(label, [pattern])  # type: ignore


In [13]:
是否输出详细信息 = False

读取数据

In [14]:
def 提取实体(doc):
    print('AI开始思考。。。')
    return doc.ents

In [15]:
def 预测(pdf文件):
    doc = nlp(读取pdf(pdf文件))
    实体列表 = 提取实体(doc)
    证券代码 = 实体列表[0].text
    # print('证券代码=',证券代码)
    # 证券代码 = '002069'
    # 遍历doc中的所有token，将text属性值放入一个list
    输入词列表 = [token.text for token in doc if token.text.strip() != '']
    #将输入词列表写入到文件
    pdf文件名 = 获取pdf文件的文件名(pdf文件)
    with open('输入词列表' + pdf文件名 + '.txt', 'w', encoding='utf-8') as f:
        f.write('","'.join(输入词列表))
    results = session.read_transaction(run_query, 图数据库查询用语句, 证券代码,输入词列表)
    if len(results) > 0:
        return (True, 可解释信息)
    else:
        return (False, '')

In [16]:
#遍历pdf文件列表，预测每个pdf文件
for pdf文件 in pdf文件列表:
    预测结果 = 预测(pdf文件)
    print(获取pdf文件的文件名(pdf文件), ': ', 预测结果[0],'。相关可解释信息：', 预测结果[1])
    

AI开始思考。。。
2014年10月30日 关于海洋牧场灾情说明会的公告.PDF :  True 。相关可解释信息： 因为该上市公司于2014年10月30日发生过类似的事情，参考信息：https://zhuanlan.zhihu.com/p/382126195;https://www.sohu.com/a/474688580_473133
AI开始思考。。。
2018年2月05日 关于底播虾夷扇贝2017年终盘点情况的公告.PDF :  True 。相关可解释信息： 因为该上市公司于2014年10月30日发生过类似的事情，参考信息：https://zhuanlan.zhihu.com/p/382126195;https://www.sohu.com/a/474688580_473133
AI开始思考。。。
2019年11月11日 关于2019年秋季底播虾夷扇贝存量抽测的风险提示公告.PDF :  True 。相关可解释信息： 因为该上市公司于2014年10月30日发生过类似的事情，参考信息：https://zhuanlan.zhihu.com/p/382126195;https://www.sohu.com/a/474688580_473133
AI开始思考。。。
第五届董事会第十七次会议决议公告.PDF :  False 。相关可解释信息： 
